In [2]:
import geemap

In [3]:
import ee

In [4]:
ee.Authenticate()

In [5]:
ee.Initialize(project ="ee-shirwadkarsejal809")

In [6]:
m = geemap.Map()
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [9]:
India = ee.FeatureCollection("projects/ee-shirwadkarsejal809/assets/India_state")
m.addLayer(India, {}, "India")
m

Map(bottom=812.0, center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Search…

In [18]:
Karnataka = India.filter(ee.Filter.eq('STATE', 'KARNATAKA')) #To clip boundary
m.addLayer(Karnataka, {}, "Karnataka")
m

Map(bottom=2066.0, center=[0.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [12]:
Karnataka_LULC = ee.ImageCollection("ESA/WorldCover/v200").first()

In [19]:
Karnataka_clipped = Karnataka_LULC.clip(Karnataka);

In [20]:
Karnataka_vis = {
    'bands':['Map'],
};
m.centerObject(Karnataka); # To centre Karnataka on map
m.addLayer(Karnataka_clipped, Karnataka_vis,'Karnataka_LULC');
m

Map(bottom=2348.0, center=[0.0, 0.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=S…

In [21]:
landsat_9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2").filterDate('2021-01-01', '2022-01-01').filterMetadata('CLOUD_COVER','less_than',10).mean().clip(Karnataka)
landsat9 = landsat_9.select('SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7')
m.addLayer(landsat_9, {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min':9304.1,'max':16669}, 'Landsat_9')
m

Map(bottom=4140.0, center=[11.178401873711785, 78.39429068622933], controls=(WidgetControl(options=['position'…

In [23]:
palettes = {'min':0, 'max':1, 'palettes':	['#006400', '#ffbb22', '#ffff4c', '#f096ff', '#fa0000', '#b4b4b4', '#f0f0f0', '#0064c8', '#0096a0', '#00cf75', '#fae6a0']}
Legend = {'Tree cover':'#006400',
          'Shrubland':'#ffbb22',
          'Grassland':'#ffff4c',
          'Cropland':'#f096ff',
          'Built-up':'#fa0000',
          'Bare / sparse vegetation':'#b4b4b4',
          'Snow and ice':'#f0f0f0',
          'Water':'#0064c8',
          'Permanent water bodies':'#0096a0',
          'Herbaceous wetland':'#00cf75',
          'Mangroves':'#fae6a0',
          'Moss and lichen':'#fae6a0'}

In [24]:
TP = Karnataka_LULC.sample(**{
    'region':Karnataka.geometry(),
    'scale':20,
    'numPixels':3000,
    'seed':1,
    'geometries':True
    })
m.addLayer(TP,{}, 'Training_points')
m

Map(bottom=7980.0, center=[11.178401873711785, 78.39429068622933], controls=(WidgetControl(options=['position'…

In [26]:
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6']
Label = 'Map'
training = landsat_9.select(bands).sampleRegions(**{
    'collection':TP,
    'properties':[Label],
    'scale':20
    })
trained = ee.Classifier.smileCart().train(training, Label, bands)

In [27]:
result = landsat_9.select(bands).classify(trained)

In [28]:
m. addLayer(result.randomVisualizer(),{},'SU_LULC')
m

Map(bottom=15345.0, center=[14.551684056143447, 76.71651060068817], controls=(WidgetControl(options=['position…